## Data Exploration and Cleaning for Housing Data and Climate Data

### Read in the Housing Data

In [1]:
#import reqs
import pandas as pd
import numpy as np
import openpyxl

In [2]:
zillow_data = pd.read_csv('housing_data/county_market_data_zillow.csv',encoding= 'unicode_escape')
zillow_data.head(3)
# zillow data is formatted by county then state and then home prices to the x axis with year-month-day format

,RegionID,SizeRank,RegionName,RegionType,§,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,1/31/00,...,10/31/22,11/30/22,12/31/22,1/31/23,2/28/23,3/31/23,4/30/23,5/31/23,6/30/23,7/31/23
0,3101,0,Los Angeles County,county,CA,CA,"Los Angeles-Long Beach-Anaheim, CA",6,37,205982.2441,...,841135.9251,838631.5389,835366.3072,828060.8150,818172.4579,809992.2976,806193.0008,808305.1445,815021.4661,826958.4611
1,139,1,Cook County,county,IL,IL,"Chicago-Naperville-Elgin, IL-IN-WI",17,31,136354.9163,...,284950.8576,284143.6694,283375.2762,284008.8916,284769.8440,286109.7605,286873.5957,287996.2025,290010.5949,292281.6116
2,1090,2,Harris County,county,TX,TX,"Houston-The Woodlands-Sugar Land, TX",48,201,107050.2397,...,283073.6242,282597.1860,281442.8051,279727.9624,278170.0439,277582.1914,277509.8971,277852.7056,278500.8659,279400.6745


In [3]:
# get view of column names for copy/pasting ease
column_names = (zillow_data.columns).tolist()

In [4]:
# check for duplicate rows if region name and state name match
# this value should come out as zero which means distinct groups
count_duplicate_rows = zillow_data.duplicated(subset=['RegionName', 'State']).sum()
count_duplicate_rows

0

### Read in the state codes to decipher the initial column in all climate data

In [5]:
# Read the state_codes Excel file
state_codes_df = pd.read_csv('climate_data/decipher/state_codes.csv')
county_fips_code = pd.read_csv('climate_data/decipher/county_fips_code.csv')
county_to_climdivs = pd.read_csv('climate_data/decipher/county-to-climdivs.csv')
county_to_climdivs.head(3)

,POSTAL_FIPS_ID,NCDC_FIPS_ID,CLIMDIV_ID
0,1033,1033,101
1,1059,1059,101
2,1077,1077,101


In [6]:
# function to split up the fips code
def split_number(x):
    x_str = str(x)
    if len(x_str) == 4:
        return x_str[0], x_str[1:]
    elif len(x_str) == 5:
        return x_str[:2], x_str[2:]

# Apply the function to split numbers and store them in new columns
county_fips_code[['State Code', 'FIPS Code']] = county_fips_code['FIPS code'].apply(lambda x: pd.Series(split_number(x)))
county_fips_code.head(3)

,FIPS code,name,State Code,FIPS Code
0,1000,Alabama,1,000
1,1001,Autauga County,1,001
2,1003,Baldwin County,1,003


In [7]:
# change data type for rows
county_fips_code['State Code'] = county_fips_code['State Code'].astype(str)
state_codes_df['ActualDivCode'] = state_codes_df['ActualDivCode'].astype(str)

# match the rows and see if they are in each csv
matching_rows = county_fips_code[county_fips_code['State Code'].isin(state_codes_df['ActualDivCode'])]

# state codes to be used later
result = state_codes_df.merge(county_fips_code, left_on='ActualDivCode', right_on='State Code', how='right')
last_col = result.columns[-1]
state_codes = result.drop(last_col, axis=1)
state_codes.head(3)

,ClimDivCode,ActualDivCode,StateName,FIPS code,name,State Code
0,1.0,1,Alabama,1000,Alabama,1
1,1.0,1,Alabama,1001,Autauga County,1
2,1.0,1,Alabama,1003,Baldwin County,1


In [8]:
county_to_climdivs.rename(columns={'POSTAL_FIPS_ID': 'FIPS code'}, inplace=True)
county_to_climdivs.head(3)

,FIPS code,NCDC_FIPS_ID,CLIMDIV_ID
0,1033,1033,101
1,1059,1059,101
2,1077,1077,101


In [9]:
# rename column in county
county_to_climdivs.rename(columns={'POSTAL_FIPS_ID': 'FIPS code'}, inplace=True)

# perform left merge on state_codes and county_to_climdivs on FIPS code to get 
# postal code and everything else correct
state_codes_merged = state_codes.merge(county_to_climdivs, left_on = 'FIPS code',right_on = 'FIPS code', how = 'right')
state_codes_merged.head(3)

,ClimDivCode,ActualDivCode,StateName,FIPS code,name,State Code,NCDC_FIPS_ID,CLIMDIV_ID
0,1.0,1,Alabama,1033,Colbert County,1,1033,101
1,1.0,1,Alabama,1059,Franklin County,1,1059,101
2,1.0,1,Alabama,1077,Lauderdale County,1,1077,101


### Read in the Climate Data

In [10]:
# read in all climate data
precip = pd.read_csv('climate_data/precipitation.csv')
temp_min = pd.read_csv('climate_data/tmin.csv')
temp_max = pd.read_csv('climate_data/tmax.csv')
temp_avg = pd.read_csv('climate_data/tempavg.csv')
cooling_days = pd.read_csv('climate_data/cddccy.csv')
heating_days = pd.read_csv('climate_data/hddccy.csv')
precip.head(3)

,date,jan,feb,mar,apr,may,jun,jul,aug,sept,oct,nov,dec
0,1001011895,7.03,2.96,8.36,3.53,3.96,5.40,3.92,3.36,0.73,2.03,1.44,3.66
1,1001011896,5.86,5.42,5.54,3.98,3.77,6.24,4.38,2.57,0.82,1.66,2.89,1.94
2,1001011897,3.27,6.63,10.94,4.35,0.81,1.57,3.96,5.02,0.87,0.75,1.84,4.38


In [11]:
# helper function to split the data string into individual pieces
def split_date(s):
    pieces = []
    if len(s) == 10:
        pieces.append(s[-4:])
        pieces.append(s[-6:-4])
        pieces.append(s[-9:-6])
        pieces.append(s[-0])
    elif len(s) == 11:
        pieces.append(s[-4:])
        pieces.append(s[-6:-4])
        pieces.append(s[-9:-6])
        pieces.append(s[-11:-9])
    else:
        print("Invalid String")
    return pieces[::-1] # reverse the string return original order

In [12]:
# use helper function split date to split the data in each csv into 
# individual components then make those into their own columns
def apply_split_date(df, date_column):
    df['date_str'] = df[date_column].astype(str)
    df['split_date'] = df['date_str'].apply(split_date)
    df[['ClimDivCode', 'fips_code', 'weather_data_type', 'year']] = pd.DataFrame(df['split_date'].tolist(), index=df.index)
    df.drop(['date_str', 'split_date'], axis=1, inplace=True)
    cols = df.columns.tolist()
    df = df[cols[-4:] + cols[:-4]]
    return df

# make new dataframes with split data
precipitation = apply_split_date(precip, 'date').drop('date', axis = 1)
precipitation['NCDC_FIPS_ID'] = precipitation['ClimDivCode'] + precipitation['fips_code']

temperature_min = apply_split_date(temp_min, 'date').drop('date', axis = 1)
temperature_min['NCDC_FIPS_ID'] = temperature_min['ClimDivCode'] + temperature_min['fips_code']

temperature_max = apply_split_date(temp_max, 'date').drop('date', axis = 1)
temperature_max['NCDC_FIPS_ID'] = temperature_max['ClimDivCode'] + temperature_max['fips_code']

temperature_avg = apply_split_date(temp_avg, 'date').drop('date', axis = 1)
temperature_avg['NCDC_FIPS_ID'] = temperature_avg['ClimDivCode'] + temperature_avg['fips_code']

num_cooling_days = apply_split_date(cooling_days, 'date').drop('date', axis = 1)
num_cooling_days['NCDC_FIPS_ID'] = num_cooling_days['ClimDivCode'] + num_cooling_days['fips_code']

num_heating_days = apply_split_date(heating_days, 'date').drop('date', axis = 1)
num_heating_days['NCDC_FIPS_ID'] = num_heating_days['ClimDivCode'] + num_heating_days['fips_code']

In [13]:
# replace all values that are 999 (invalid data) with N/A
precipitation = precipitation.replace(-9.99, np.nan)
temperature_min = temperature_min.replace(-99.9, np.nan)
temperature_max = temperature_max.replace(-99.9, np.nan)
temperature_avg = temperature_avg.replace(-99.9, np.nan)
num_cooling_days = num_cooling_days.replace(-9999, np.nan)
num_heating_days = num_heating_days.replace(-9999, np.nan)
len(precipitation)

403773

In [14]:
# merge the state code data to each climate dataframe with information 
# convert state_codes to int type for ease
state_codes_merged['NCDC_FIPS_ID'] = state_codes_merged['NCDC_FIPS_ID'].astype(int)

# convert each climate data to in for ease
precipitation['NCDC_FIPS_ID'] = precipitation['NCDC_FIPS_ID'].astype(int)
temperature_min['NCDC_FIPS_ID'] = temperature_min['NCDC_FIPS_ID'].astype(int)
temperature_max['NCDC_FIPS_ID'] = temperature_max['NCDC_FIPS_ID'].astype(int)
temperature_avg['NCDC_FIPS_ID'] = temperature_avg['NCDC_FIPS_ID'].astype(int)
num_cooling_days['NCDC_FIPS_ID'] = num_cooling_days['NCDC_FIPS_ID'].astype(int)
num_heating_days['NCDC_FIPS_ID'] = num_heating_days['NCDC_FIPS_ID'].astype(int)

# preform multiple merges for new data tables with state codes
preciptation_state = pd.merge(precipitation, state_codes_merged, on=['NCDC_FIPS_ID'])
temperature_min_state = pd.merge(temperature_min, state_codes_merged, on=['NCDC_FIPS_ID'])
temperature_max_state = pd.merge(temperature_max, state_codes_merged, on=['NCDC_FIPS_ID'])
temperature_avg_state = pd.merge(temperature_avg, state_codes_merged, on=['NCDC_FIPS_ID'])
num_cooling_days_state = pd.merge(num_cooling_days, state_codes_merged, on=['NCDC_FIPS_ID'])
num_heating_days_state = pd.merge(num_heating_days, state_codes_merged, on=['NCDC_FIPS_ID'])

In [17]:
# check data length and percent loss
class data_checks:
    @staticmethod
    #Check the length of each dataframe
    def check_length_df(*dfs):
        for i, df in enumerate(dfs):
            print(f"Length of df {i+1}: {len(df)}")
            
    # Check the percentage loss of each dataframe from the original
    @staticmethod
    def data_loss(*dfs, expected_length=403773):
        for i, df in enumerate(dfs):
            loss = expected_length - len(df)
            percent_loss = (loss / expected_length) * 100
            print(f"Data loss {i+1}: {percent_loss:.2f}%")


In [18]:
data_checks.check_length_df(preciptation_state, temperature_min_state, temperature_max_state, temperature_avg_state, 
                  num_cooling_days_state, num_heating_days_state)

Length of df 1: 399771
Length of df 2: 399771
Length of df 3: 399771
Length of df 4: 399771
Length of df 5: 399771
Length of df 6: 399771


In [19]:
data_checks.data_loss(preciptation_state, temperature_min_state, temperature_max_state, temperature_avg_state, 
                  num_cooling_days_state, num_heating_days_state)

Data loss 1: 0.99%
Data loss 2: 0.99%
Data loss 3: 0.99%
Data loss 4: 0.99%
Data loss 5: 0.99%
Data loss 6: 0.99%


#### There appears to be data loss in the climate data when matching to the FIPS code which means that not all FIPS code align with a county, after looking back at the readme file for the climate data it appears that it aligns with a certain geographical region which does not always align with a specific FIPS code county, that and HI and Alaska do not appear. Hence data loss occured but for this it should be fine to continue on with analysis. More data can always be added later to improve the model

#### EDIT: fixed the data loss issue and added in data checks now reading in at less than 1% data loss which is fine, probably just n/a data that should be removed

In [20]:
# save all dfs to csv for building out SQL database

preciptation_state.to_csv('preciptation_state.csv')
temperature_min_state.to_csv('temperature_min_state.csv')
temperature_max_state.to_csv('temperature_max_state.csv')
temperature_avg_state.to_csv('temperature_avg_state.csv')
num_cooling_days_state.to_csv('num_cooling_days_state.csv')
num_heating_days_state.to_csv('num_heating_days_state.csv')